In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1698851213873_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
PATH = 's3://workspace8'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://workspace8
PATH_Data:   s3://workspace8/Test
PATH_Result: s3://workspace8/Results

In [4]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://workspace8/T...|2023-11-01 16:58:02|  6574|[FF D8 FF E0 00 1...|
|s3://workspace8/T...|2023-11-01 16:57:41|  6573|[FF D8 FF E0 00 1...|
|s3://workspace8/T...|2023-11-01 16:57:05|  6573|[FF D8 FF E0 00 1...|
|s3://workspace8/T...|2023-11-01 16:57:36|  6572|[FF D8 FF E0 00 1...|
|s3://workspace8/T...|2023-11-01 16:57:10|  6572|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [6]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------+---------+
|path                                      |label    |
+------------------------------------------+---------+
|s3://workspace8/Test/Pineapple/99_100.jpg |Pineapple|
|s3://workspace8/Test/Pineapple/289_100.jpg|Pineapple|
|s3://workspace8/Test/Pineapple/326_100.jpg|Pineapple|
|s3://workspace8/Test/Pineapple/143_100.jpg|Pineapple|
|s3://workspace8/Test/Pineapple/243_100.jpg|Pineapple|
+------------------------------------------+---------+
only showing top 5 rows

None

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+---------+
|                path|   modificationTime|length|             content|    label|
+--------------------+-------------------+------+--------------------+---------+
|s3://workspace8/T...|2023-11-01 16:58:02|  6574|[FF D8 FF E0 00 1...|Pineapple|
|s3://workspace8/T...|2023-11-01 16:57:41|  6573|[FF D8 FF E0 00 1...|Pineapple|
|s3://workspace8/T...|2023-11-01 16:57:05|  6573|[FF D8 FF E0 00 1...|Pineapple|
|s3://workspace8/T...|2023-11-01 16:57:36|  6572|[FF D8 FF E0 00 1...|Pineapple|
|s3://workspace8/T...|2023-11-01 16:57:10|  6572|[FF D8 FF E0 00 1...|Pineapple|
+--------------------+-------------------+------+--------------------+---------+
only showing top 5 rows

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------+---------+
|path                                      |label    |
+------------------------------------------+---------+
|s3://workspace8/Test/Pineapple/99_100.jpg |Pineapple|
|s3://workspace8/Test/Pineapple/289_100.jpg|Pineapple|
|s3://workspace8/Test/Pineapple/326_100.jpg|Pineapple|
|s3://workspace8/Test/Pineapple/143_100.jpg|Pineapple|
|s3://workspace8/Test/Pineapple/243_100.jpg|Pineapple|
+------------------------------------------+---------+
only showing top 5 rows

None

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1698851213873_0003/container_1698851213873_0003_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [15]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df.select('path').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                path|
+--------------------+
|s3://workspace8/T...|
|s3://workspace8/T...|
|s3://workspace8/T...|
|s3://workspace8/T...|
|s3://workspace8/T...|
+--------------------+
only showing top 5 rows

In [18]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://workspace8/Results

In [19]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df = pd.read_parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                         path  ...                                           features
0  s3://workspace8/Test/Pineapple/189_100.jpg  ...  [0.0, 3.5926237, 0.08994417, 0.0, 0.0, 0.02846...
1  s3://workspace8/Test/Pineapple/253_100.jpg  ...  [0.0, 4.228983, 0.065928906, 0.0, 0.0, 0.02050...
2  s3://workspace8/Test/Pineapple/307_100.jpg  ...  [0.0, 4.5849614, 0.010436758, 0.0, 0.042496923...
3   s3://workspace8/Test/Pineapple/16_100.jpg  ...  [0.0, 4.842381, 0.02923479, 0.0, 0.0, 0.0, 0.0...
4  s3://workspace8/Test/Pineapple/236_100.jpg  ...  [0.0, 4.1937304, 0.11984043, 0.0, 0.0, 0.12534...

[5 rows x 3 columns]